In [1]:
import pandas as pd
import numpy as np

In [2]:
MML_VARIABLE_ANALYZED = 'TLF'
COUNTRY_COLUMN = 'ref_area.label'
regions_countries = {
    'LAM':['Argentina',
           'Bolivia',
           'Brazil',
          'Chile',
          'Colombia',
          'Costa Rica',
          'Cuba',
          'Ecuador',
          'El Salvador',
          'Guatemala',
          'Haiti',
          'Honduras',
          'Jamaica',
          'Mexico',
          'Nicaragua',
          'Panama',
          'Paraguay',
          'Peru',
          'Dominican Republic',
          'Trinidad and Tobago',
          'Uruguay',
          'Venezuela']
}

In [3]:
df = pd.read_excel('data/EAP_TEAP_SEX_AGE_NB_A-filtered-2022-08-02.xlsx', sheet_name='EAP_TEAP_SEX_AGE_NB_A-filtered-')
#Annual Total Population at Mid-Year (thousands)

In [4]:
df.columns = df.columns.astype(str)
df = df[[COUNTRY_COLUMN, 'time', 'obs_value']]
df[COUNTRY_COLUMN]= df[COUNTRY_COLUMN].str.strip()

In [5]:
df[0:5]

,ref_area.label,time,obs_value
0,Aruba,2007,54729
1,Aruba,2011,52585
2,Aruba,1991,31.11
3,Aruba,2010,52045
4,Afghanistan,2008,7678092


In [6]:
def get_countries_for_region(df, country_column, region='LAM'):
    return df.loc[df[country_column].isin(regions_countries[region])]
def get_missing_countries_for_region(df, country_column, region='LAM'):
    df_countries_list = get_countries_for_region(df, country_column, region)[country_column]
    return set(regions_countries[region]) - set(df_countries_list)

In [7]:
print("Missing countries for region")
print(get_missing_countries_for_region(df, COUNTRY_COLUMN))
#Actually, they are not missing, they have a different value
df.loc[df[COUNTRY_COLUMN] == 'Venezuela, Bolivarian Republic of', COUNTRY_COLUMN] = 'Venezuela'

print("Missing countries for region after fix")
print(get_missing_countries_for_region(df, COUNTRY_COLUMN))

Missing countries for region
{'Venezuela'}
Missing countries for region after fix
set()


In [11]:
get_countries_for_region(df, COUNTRY_COLUMN).sort_values([COUNTRY_COLUMN,'time'],ascending=[True,True])[26:50]

,ref_area.label,time,obs_value
419,Bolivia,1980,1822305
420,Bolivia,1981,1864.99
421,Bolivia,1982,1890.7
422,Bolivia,1985,1996459
423,Bolivia,1986,2076782
424,Bolivia,1987,2101052
401,Bolivia,1995,1316723
402,Bolivia,1996,1413228
403,Bolivia,1997,3645165
404,Bolivia,2000,3823937


In [ ]:
df_latin_america_truth = get_countries_for_region(df, COUNTRY_COLUMN)
assert len(df_latin_america_truth) == len(regions_countries['LAM']), 'MISSING COUNTRIES FOR REGION'

In [ ]:
years_filtered = np.arange(1970,2020,2) #[1970,2020) just even years
columns_filter = np.append([COUNTRY_COLUMN],years_filtered)

df_latin_america_truth = df_latin_america_truth[columns_filter]
assert 0 == df_latin_america_truth.isna().sum().sum(), "MISSING VALUES. CONSIDER FILL STRATEGY" #If 0, then we don't have missing values.

Finally, we got our truth dataframe. Now, MML

In [ ]:
df_mml = pd.read_csv('data/mml_lam_1.csv')
df_mml = df_mml[['Year',MML_VARIABLE_ANALYZED]]
df_mml = df_mml[df_mml.Year <= 2020]
# According to truth dataset, population values are expressed in thousands, so 1 in truth is actually 1000.
df_mml[MML_VARIABLE_ANALYZED] = df_mml[MML_VARIABLE_ANALYZED] / 1000 
df_mml[0:3]

Ok, that was easy. Back to truth df. 
We need to transpose df, get only even years and group by some condition

In [ ]:
df_T = df_latin_america_truth.set_index(COUNTRY_COLUMN).T
df_T = df_T.reset_index().rename(columns={"index": "Year"})
# df_T.rename_axis(None, axis=1)
# df_T.index.name = None

df_latin_america_truth = df_T.astype({'Year': 'int64'})
df_latin_america_truth

In [ ]:
df_latin_america_truth['sum'] = df_latin_america_truth.iloc[:, 1:len(regions_countries['LAM'])+1].sum(axis=1)
df_latin_america_truth_aggs = df_latin_america_truth[['Year','sum']]

In [ ]:
df = pd.merge(df_mml,df_latin_america_truth_aggs,on='Year').rename(columns={"POP": "mml_value", "sum":"truth_value"})
df[0:3]

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [18,10]

In [ ]:
def plot_lines(df):
    # plot lines
    plt.plot(df['Year'], df['mml_value'], label = "MML Values")
    plt.plot(df['Year'], df['truth_value'], label = "Truth Values")
    
    plt.xticks(df['Year'], rotation=90)
    plt.xlabel('Year')
    plt.ylabel(MML_VARIABLE_ANALYZED)
    plt.grid(axis = 'x')
    plt.legend()
    plt.show()
plot_lines(df)

### MSE
MSE is one of the most common regression loss functions. In Mean Squared Error also known as L2 loss, we calculate the error by squaring the difference between the predicted value and actual value and averaging it across the dataset. MSE is also known as Quadratic loss as the penalty is not proportional to the error but to the square of the error.

**Squaring the error gives higher weight to the outliers, which results in a smooth gradient for small errors.** 

**The value of the error ranges from zero to infinity.**

**MSE increases exponentially with an increase in error.**

**A good model will have an MSE value closer to zero.**

https://www.analyticsvidhya.com/blog/2021/10/evaluation-metric-for-regression-models/

### RMSE
Similar, but it takes square root to preserve the original magnitude of the diference in the errors.

In [ ]:
def mean_squared_error(true, pred):
    squared_error = np.square(true - pred) 
    sum_squared_error = np.sum(squared_error)
    mse_loss = sum_squared_error / true.size
    return mse_loss

def root_mean_squared_error(true, pred):
    squared_error = np.square(true - pred) 
    sum_squared_error = np.sum(squared_error)
    rmse_loss = np.sqrt(sum_squared_error / true.size)
    return rmse_loss

In [ ]:
rmse = root_mean_squared_error(df['truth_value'],df['mml_value'])
mse = mean_squared_error(df['truth_value'],df['mml_value'])


print(mse, rmse)

As we see in the graph, from 1988, mml stops growing as the truth continues the original tendence.

### 1970 - 1988

So, let's take only from 1970-1988

In [ ]:
df_1988 = df[df['Year']<=1988]
rmse_1988 = root_mean_squared_error(df_1988['truth_value'],df_1988['mml_value'])
mse_1988 = mean_squared_error(df_1988['truth_value'],df_1988['mml_value'])

print(mse_1988, rmse_1988)
plot_lines(df_1988)

Mean seems to be quite accurate in that interval

### Error visualization

We have a truth and a mml value.

Off course, we assume that truth is truth.

So the idea is calculate, year by year, the difference between them in terms of proportion. i.e 5% of difference for 1970; -3.6% of difference for 1974, etc

The sign will indicate if mml is below (-) or above (+) truth

In [ ]:
df['diff_%'] = (df['mml_value'] - df['truth_value']) / df['truth_value'] * 100
df

In [ ]:
plt.bar(df['Year'], df['diff_%'])
plt.grid(axis='y')
# plt.ylim(-50,50)
plt.yticks(np.arange(-25,26,5))
plt.xticks(years_filtered)
plt.xlabel("Year")
plt.ylabel("% difference")
plt.title(f"Porcentual difference between truth and MML for {MML_VARIABLE_ANALYZED}")
plt.show()

In [ ]:
df_diff_porcentual = df[['Year', 'diff_%']]
df_diff_porcentual.columns = ['Year', MML_VARIABLE_ANALYZED]

In [ ]:
#TODO. MEJORAR ESTO PARA QUE SE BANQUE QUE SI NO EXISTE EL ARCHIVO LO CREA, SI EXISTE SOBREESCRIBE COLUMNA.

d = pd.read_excel('data/output/diff_porcentual.xlsx')
d[MML_VARIABLE_ANALYZED] = df_diff_porcentual[MML_VARIABLE_ANALYZED]
d.to_excel('data/output/diff_porcentual.xlsx', index=False)